In [387]:
from datetime import datetime, time, timedelta, timezone;
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pmaw import PushshiftAPI;
import sqlite3;

In [388]:
class Database:
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()
        self.connection = sqlite3.connect('db')
        
        self.connection.execute('DROP TABLE IF EXISTS Subreddit')
        self.connection.commit()
        self.connection.execute('''
        CREATE TABLE IF NOT EXISTS Subreddit(
            id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            name VARCHAR NOT NULL,
            UNIQUE(name),
            CHECK(name <> '')
        )
        ''')
        
        self.connection.execute('DROP TABLE IF EXISTS Post')
        self.connection.commit()
        self.connection.execute('''
        CREATE TABLE IF NOT EXISTS Post(
            id VARCHAR PRIMARY KEY NOT NULL,
            subreddit INTEGER NOT NULL,
            title VARCHAR NOT NULL,
            body VARCHAR,
            url VARCHAR NOT NULL,
            comments INTEGER NOT NULL,
            crossposts INTEGER NOT NULL,
            score INTEGER NOT NULL,
            UNIQUE(id),
            FOREIGN KEY(subreddit) REFERENCES Subreddit(id),
            CHECK(title <> ''),
            CHECK(url <> '')
        )
        ''')
        self.connection.commit()
        
        self.connection.execute('DROP TABLE IF EXISTS VaderAnalysis')
        self.connection.commit()
        self.connection.execute('''
        CREATE TABLE IF NOT EXISTS VaderAnalysis(
            id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            related_id VARCHAR NOT NULL,
            related_type VARCHAR NOT NULL,
            related_attribute VARCHAR NOT NULL,
            negative DOUBLE NOT NULL,
            neutral DOUBLE NOT NULL,
            positive DOUBLE NOT NULL,
            compound DOUBLE NOT NULL,
            UNIQUE(related_id, related_type, related_attribute)
        )
        ''')
        self.connection.commit()
        
        self.connection.execute('DROP TABLE IF EXISTS SymbolType')
        self.connection.commit()
        self.connection.execute('''
        CREATE TABLE IF NOT EXISTS SymbolType(
            id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            type VARCHAR NOT NULL,
            UNIQUE(type)
        )
        ''')
        self.connection.commit()
        
        self.connection.execute('INSERT INTO SymbolType (type) VALUES (?)', ['Stock'])
        self.connection.commit()
        self.connection.execute('INSERT INTO SymbolType (type) VALUES (?)', ['Mutual Fund'])
        self.connection.commit()
        
        
        self.connection.execute('DROP TABLE IF EXISTS Symbol')
        self.connection.commit()
        self.connection.execute('''
        CREATE TABLE IF NOT EXISTS Symbol(
            id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            symbol VARCHAR NOT NULL,
            symbol_type INTEGER NOT NULL,
            FOREIGN KEY(symbol_type) REFERENCES SymbolType(id),
            UNIQUE(symbol, symbol_type)
        )
        ''')
        self.connection.commit()
        
        self.connection.execute('DROP TABLE IF EXISTS PostSymbol')
        self.connection.commit()
        self.connection.execute('''
        CREATE TABLE IF NOT EXISTS PostSymbol(
            id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            post_id INTEGER NOT NULL,
            symbol_id INTEGER NOT NULL,
            FOREIGN KEY(post_id) REFERENCES Post(id),
            FOREIGN KEY(symbol_id) REFERENCES Symbol(id),
            UNIQUE(post_id, symbol_id)
        )
        ''')
        self.connection.commit()
        
        
    def get_vader_analysis(self, related_id = None, related_type = None, related_attribute = None):
        if related_id is None:
            return None
        
        if related_type is None:
            return None
        
        if related_attribute is None:
            return None
        
        return self.connection.execute('SELECT * FROM VaderAnalysis WHERE related_id = ? AND related_type = ? AND related_attribute = ?', [related_id, related_type, related_attribute]).fetchone()
        
    def get_subreddit(self, subreddit_name = None):
        if subreddit_name is None:
            return None
        
        return self.connection.execute('SELECT * FROM Subreddit WHERE name = ?', [subreddit_name]).fetchone()
    
    def get_post(self, post = None):
        if post is None:
            return None
        
        return self.connection.execute('SELECT * FROM Post WHERE id = ?', [post['id']]).fetchone()
    
    def get_symbol_id(self, symbol = None):
        if symbol is None:
            return None
        
        if symbol == '':
            return None
        
        return self.connection.execute('SELECT id FROM Symbol WHERE symbol = ?', [symbol]).fetchone()[0]
    
    def insert_vader_analysis_for_post(self, post = None):
        if post is None:
            return
        
        if self.get_vader_analysis(post['id'], 'Post') is None:
            scores = self.analyzer.polarity_scores(post['title'])
            
            values = [
                post['id'],
                'Post',
                'Title',
                scores['neg'],
                scores['neu'],
                scores['pos'],
                scores['compound']
            ]
            self.connection.execute('''
            INSERT INTO VaderAnalysis (related_id, related_type, related_attribute, negative, neutral, positive, compound)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', values)
            self.connection.commit()
            
            if post['selftext'] == '':
                return
            
            scores = self.analyzer.polarity_scores(post['selftext'])

            values = [
                post['id'],
                'Post',
                'Body',
                scores['neg'],
                scores['neu'],
                scores['pos'],
                scores['compound']
            ]
            self.connection.execute('''
            INSERT INTO VaderAnalysis (related_id, related_type, related_attribute, negative, neutral, positive, compound)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', values)
            self.connection.commit()
        
    def insert_vader_analysis_for_comment(self, comment = None):
        if comment is None:
            return
    
    def insert_subreddit(self, subreddit_name = None):
        if subreddit_name is None:
            return
        
        self.connection.execute('INSERT OR IGNORE INTO Subreddit (name) VALUES (?)', [subreddit_name])
        self.connection.commit()
        
    def insert_post(self, post = None):
        if post is None:
            return
        
        if self.get_post(post) is not None:
            return
        
        if self.get_subreddit(post['subreddit']) is None:
            self.insert_subreddit(post['subreddit'])
            
        values = [
            post['id'],
            self.get_subreddit(post['subreddit'])[0],
            post['title'],
            post['selftext'] if 'selftext' in post else '',
            post['full_link'],
            post['num_comments'],
            post['num_crossposts'],
            post['score']
        ]
            
        self.connection.execute('''
        INSERT INTO Post (id, subreddit, title, body, url, comments, crossposts, score)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', values)
        self.connection.commit()
        
        self.insert_vader_analysis_for_post(post)
        
    def insert_symbols(self, symbols = None, symbol_type = None):
        if symbols is None or type is None:
            return
        
        symbols = list(map(lambda symbol: (symbol.strip(), symbol_type), symbols))
        
        self.connection.executemany('INSERT OR IGNORE INTO Symbol (symbol, symbol_type) VALUES (?, ?)', symbols)
        self.connection.commit()
        
    def insert_post_symbols(self, post_id = None, symbols = None):
        if post_id is None or symbols is None:
            return
        
        symbols = list(map(lambda symbol: (self.get_symbol_id(symbol), post_id), symbols))
        
        for symbol in symbols: # No clue how it's possible for there to be a symbol that isn't in the DB, but w/e for now
            if symbol[0] is None:
                symbols.remove(symbol)
        
        if len(symbols) > 0:
            self.connection.executemany('INSERT OR IGNORE INTO PostSymbol (symbol_id, post_id) VALUES (?, ?)', symbols)
            self.connection.commit()
    
database = Database()  

In [389]:
class Post:
    def __init__(self, post = None):
        if post is None:
            raise 'You must specify a post.'
        
        self.created_yesterday = None
        self.post = post
        self.symbols = None
        
        
    
    def created_yesterday(self):
        if self.created_yesterday is not None:
            return self.created_yesterday
        
        
        yesterday = datetime.now(timezone.utc) - timedelta(days=1)
        
        start_of_yesterday = datetime.combine(yesterday, time.min, tzinfo=timezone.utc).timestamp()
        end_of_yesterday = datetime.combine(yesterday, time.max, tzinfo=timezone.utc).timestamp()
        
        self.created_yesterday = start_of_yesterday <= created_utc <= end_of_yesterday
    
    
    
    def detected_symbols(self, symbols = None):
        if self.symbols is not None:
            return self.symbols
        
        if symbols is None:
            return []
        
        if len(symbols) == 0:
            return []
        
        self.symbols = []
        
        if 'title' in self.post:
            for symbol in symbols:
                if symbol in self.post['title']:
                    self.symbols.append(symbol)
                
        if 'selftext' in self.post:
            for symbol in symbols:
                if symbol in self.post['selftext']:
                    self.symbols.append(symbol)
                    
        return self.symbols
    
    
    
    def has_body(self):
         return 'selftext' in self.post
    
    
    
    def was_deleted(self):
        if 'selftext' not in self.post:
            return False
        
        return self.post['selftext'] == '[Deleted]'
    
    
    
    def was_removed(self):
        if 'selftext' not in self.post:
            return False
        
        return self.post['selftext'] == '[removed]'

In [390]:
# Load all symbols
mutual_fund_symbols = []
stock_symbols = []

with open('data/mfundslist.txt', 'r') as file:
    fund_symbols = [line for line in file]

with open('data/nasdaqlisted.txt', 'r') as file:
    stock_symbols = [line for line in file]
    
# Sanitize all symbols
mutual_fund_symbols = list(map(lambda symbol: symbol.strip(), mutual_fund_symbols))
stock_symbols = list(map(lambda symbol: symbol.strip(), stock_symbols))

# Add all symbols to DB
database.insert_symbols(stock_symbols, 1)
database.insert_symbols(mutual_fund_symbols, 2)

# Combine all symbols into a single array
symbols = mutual_fund_symbols + stock_symbols
mutual_fund_symbols = None
stock_symbols = None

In [391]:
# Retrieve Posts
api = PushshiftAPI()
posts = api.search_submissions(
    subreddit='stocks',
    search_window=1,
    limit=1000
)

# Convert to Post objects
posts = list(map(lambda post: Post(post), posts))

Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.


In [393]:
# Filter out removed and deleted posts, and those without a body
for post in posts:
    if not post.has_body:
        posts.remove(post)
        continue
    
    if post.was_deleted:
        posts.remove(post)
        continue
    
    if post.was_removed:
        posts.remove(post)

# Filter out posts that do not contain any symbols
for post in posts:
    if len(post.detected_symbols(symbols)) == 0:
        posts.remove(post)
        
# Add posts and post symbols to DB
for post in posts:
    database.insert_post(post.post)
    database.insert_post_symbols(post.post['id'], post.detected_symbols(symbols))

In [394]:
print(sqlite3.connect('db').execute('SELECT COUNT(*) FROM Subreddit').fetchone())
print(sqlite3.connect('db').execute('SELECT COUNT(*) FROM Post').fetchone())
print(sqlite3.connect('db').execute('SELECT COUNT(*) FROM VaderAnalysis').fetchone())
print(sqlite3.connect('db').execute('SELECT COUNT(*) FROM SymbolType').fetchone())
print(sqlite3.connect('db').execute('SELECT COUNT(*) FROM Symbol').fetchone())
print(sqlite3.connect('db').execute('SELECT COUNT(*) FROM PostSymbol').fetchone())

(1,)
(51,)
(102,)
(2,)
(5701,)
(109,)


In [395]:
from pprint import pprint
pprint(sqlite3.connect('db').execute('SELECT Post.id, GROUP_CONCAT(Symbol.symbol) AS "Detected Symbols" FROM Post JOIN PostSymbol ON Post.id = PostSymbol.post_id JOIN Symbol ON PostSymbol.symbol_id = Symbol.id GROUP BY Post.id LIMIT 20').fetchall())
print()
pprint(sqlite3.connect('db').execute('SELECT Post.id, VaderAnalysis.compound FROM Post JOIN VaderAnalysis WHERE VaderAnalysis.compound > 0 LIMIT 20').fetchall())
print()
pprint(sqlite3.connect('db').execute('SELECT Post.id, Post.title, Post.body, GROUP_CONCAT(Symbol.symbol) AS "Detected Symbols", VaderAnalysis.compound FROM Post JOIN PostSymbol ON Post.id = PostSymbol.post_id JOIN Symbol ON PostSymbol.symbol_id = Symbol.id JOIN VaderAnalysis WHERE VaderAnalysis.compound >= 0.55 GROUP BY Post.id LIMIT 20').fetchall())

[('yinpx5', 'NA'),
 ('yiqbwh', 'APCX,PCX'),
 ('yishgj', 'OM'),
 ('yj0ok0', 'LI,ME,VR,Z'),
 ('yj41ow', 'CD,TA'),
 ('yj5ewt', 'DOCU'),
 ('yj6qof', 'UG'),
 ('yj87bk', 'ANY'),
 ('yj9u8n', 'APP'),
 ('yjdfee', 'AY,EM,EMB,ON,OP,OPEN,PSL,TH'),
 ('yjfigl', 'CID,FV,LCID,TSL,TSLA'),
 ('yjgr6q', 'HA,TER,TW'),
 ('yjh7bu', 'GAN,TGAN'),
 ('yjkfoj', 'TW'),
 ('yjm52h', 'CME,ME'),
 ('yjmsdk', 'OM'),
 ('yjojzz', 'CLOU'),
 ('yjpk79', 'BRQS'),
 ('yjrx66', 'EA,EM,ME'),
 ('yjsdhk', 'ROKU')]

[('yijkne', 0.8292),
 ('yik0ba', 0.8292),
 ('yimqzf', 0.8292),
 ('yimtij', 0.8292),
 ('yinpx5', 0.8292),
 ('yinrvw', 0.8292),
 ('yiotx7', 0.8292),
 ('yiqbwh', 0.8292),
 ('yishgj', 0.8292),
 ('yiur12', 0.8292),
 ('yivpc1', 0.8292),
 ('yiyc8n', 0.8292),
 ('yj0ok0', 0.8292),
 ('yj1s96', 0.8292),
 ('yj41ow', 0.8292),
 ('yj5ewt', 0.8292),
 ('yj6lfm', 0.8292),
 ('yj6qof', 0.8292),
 ('yj7j9q', 0.8292),
 ('yj87bk', 0.8292)]

[('yinpx5',
  'VNUE in order to list onto the NADSAQ Stock Market over the coming months.',
  '[removed]'

In [357]:
database.insert_post(post_list[0])
print(database.get_post(post_list[0]))
print(database.get_subreddit(post_list[0]['subreddit']))
print(database.get_vader_analysis(post_list[0]['id'], 'Post', 'Title'))
print(database.get_vader_analysis(post_list[0]['id'], 'Post', 'Body'))

('yjlt3k', 1, 'Help! First time buying a Put', '', 'https://www.reddit.com/r/wallstreetbets/comments/yjlt3k/help_first_time_buying_a_put/', 1, 0, 1)
(1, 'wallstreetbets')
(390, 'yjlt3k', 'Post', 'Title', 0.0, 0.626, 0.374, 0.4574)
None


In [113]:
print(sqlite3.connect('db').execute('SELECT * FROM Subreddit').fetchall())
print(sqlite3.connect('db').execute('SELECT * FROM Post').fetchall())
print(sqlite3.connect('db').execute('SELECT * FROM VaderAnalysis').fetchall())
print(len(sqlite3.connect('db').execute('SELECT * FROM Post WHERE id = ?', ['yiqpy5']).fetchall()))

[(1, 'wallstreetbets')]
[('yikfga', 1, 'New to Stocks and I have $1000 to spare. Any ideas?', '[removed]', 'https://www.reddit.com/r/wallstreetbets/comments/yikfga/new_to_stocks_and_i_have_1000_to_spare_any_ideas/', 0, 0, 1)]
[]
0
